In [2]:
import tensorflow as tf
import numpy as np

tf.random.set_seed(777)

In [10]:
# 동물의 특징 16개를 가지는 dataset, 데이터 나누는 기준은 ,
xy = np.loadtxt('data-04-zoo.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:, -1]

nb_classes = 7  # 0 ~ 6

# one-hot encoding
'''
# 이전 코드
Y_one_hot = tf.one_hot(list(y_data), nb_classes)
Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes])
# one_hot()을 거치면 rank가 N -> N+1로 올라감(데이터를 한번 더 감쌈)
'''
Y_one_hot = tf.one_hot(y_data.astype(np.int32), nb_classes)

print(x_data.shape, Y_one_hot.shape)

(101, 16) (101, 7)


In [4]:
W = tf.Variable(tf.random.normal((16, nb_classes)), name='weight')
b = tf.Variable(tf.random.normal((nb_classes,)), name='bias')
variables = [W, b]

In [11]:
def logit_fn(X):
    return tf.matmul(X, W) + b

def hypothesis(X):
    return tf.nn.softmax(logit_fn(X))

# logit과 hypothesis를 분리한 이유 : cross entropy 함수를 사용하기 위함.
# hypothesis 굳이 사용할 이유가 없는데(함수에서 해주기 때문) 쓴 이유 : prediction에서 사용

def cost_fn(X, Y):
    logits = logit_fn(X)
    # 근데 버전 업그레이드로 인해 다른 함수를 사용, 추가 조사 필요.
    # 이도 강의에 나온 함수 처럼 logits을 사용하고 이를 from_logits로 표현하고 있음.
    cost_i = tf.keras.losses.categorical_crossentropy(
        y_true=Y, y_pred=logits, from_logits=True)
    cost = tf.reduce_mean(cost_i)
    return cost

def grad_fn(X, Y):
    with tf.GradientTape() as tape:
        loss = cost_fn(X, Y)
        grads = tape.gradient(loss, variables)
        return grads

def prediction(X, Y):
    pred = tf.argmax(hypothesis(X), 1)
    correct_prediction = tf.equal(pred, tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    return accuracy

In [12]:
def fit(X, Y, epochs=1000, verbose=100):
    optimizer =  tf.keras.optimizers.SGD(learning_rate=0.1)

    for i in range(epochs):
        grads = grad_fn(X, Y)
        optimizer.apply_gradients(zip(grads, variables))
        if (i == 0) | ((i + 1) % verbose == 0):
            acc = prediction(X, Y).numpy()
            loss = cost_fn(X, Y).numpy()
            print('Steps: {} Loss: {}, Acc: {}'.format(i+1, loss, acc))

fit(x_data, Y_one_hot)

Steps: 1 Loss: 0.09009025990962982, Acc: 1.0
Steps: 100 Loss: 0.08323754370212555, Acc: 1.0
Steps: 200 Loss: 0.07733815908432007, Acc: 1.0
Steps: 300 Loss: 0.07224830985069275, Acc: 1.0
Steps: 400 Loss: 0.06780780106782913, Acc: 1.0
Steps: 500 Loss: 0.06389689445495605, Acc: 1.0
Steps: 600 Loss: 0.0604240745306015, Acc: 1.0
Steps: 700 Loss: 0.05731821060180664, Acc: 1.0
Steps: 800 Loss: 0.054522931575775146, Acc: 1.0
Steps: 900 Loss: 0.0519932396709919, Acc: 1.0
Steps: 1000 Loss: 0.04969245567917824, Acc: 1.0
